In [8]:
import rdflib
import os, sys

text = sys.argv[1]

def load_graph():
    g = rdflib.Graph()
    g.parse("onto.nt", format="nt")
    return g

def check_in_ontology(g, a, b):
    q = "select * where {<http://o.org/"+a+"> ?p <http://o.org/"+b+">}"
    x1 = g.query(q)
    for x in x1:
        print()
        print(a,x,b)
        get_type(g, a)
        get_type(g, b)
    
    return x1


def get_type(g, a):
    q = "select ?t where {<http://o.org/"+a+"> <http://o.org/type> ?t}"
    x1 = g.query(q)
    for x in x1:
        print(a,x)


def normalize_word(w):
    # TODO: stemming
    return w.replace(",","").replace(".","").strip()
    
def check_text(g, text):
    text = text.split()
    for t1 in text:
        t1 = normalize_word(t1)
        for t2 in text:
            t2 = normalize_word(t2)
            if t1 != t2:
                check_in_ontology(g, t1, t2)
                
def geographic_difference(g, s, t):
    # Currently - equal weight for every connection
    s = s.lower().replace(" ", "_").split(",")[0].strip()
    t = t.lower().replace(" ", "_").split(",")[0].strip()
    if s == t:
        return 0
    return 3 - min(len(check_in_ontology(g, s, t)), 3)

def month_to_num(m):
    if m == "jan":
        return 1
    if m == "feb":
        return 2
    if m == "mar":
        return 3
    if m == "apr":
        return 4
    if m == "may":
        return 5
    if m == "jun":
        return 6
    if m == "jul":
        return 7
    if m == "aug":
        return 8
    if m == "sep":
        return 9
    if m == "oct":
        return 10
    if m == "nov":
        return 11
    if m == "dec":
        return 12
    return 0
    

def date_difference(s, t):
    print(s)
    print(t)
    if s is None or t is None:
        return 0
    s_splitted = s.lower().split()
    t_splitted = t.lower().split()
    t_d = s_d = 0
    s_m = month_to_num(s_splitted[0][0:3])
    if len(s_splitted) > 1:
        s_d = s_splitted[1]
    t_m = month_to_num(t_splitted[0][0:3])
    if len(t_splitted) > 1:
        t_d = t_splitted[1]
    
    #print(s_m, s_d, t_m, t_d)
    ret = 0
    try:
        ret = abs((s_m*30+int(s_d)) - (t_m*30+int(t_d)))
    except:
        pass
    return ret


g = load_graph()
print(geographic_difference(g, "paris", "london"))



paris (rdflib.term.URIRef('http://o.org/near_by'),) london

paris (rdflib.term.URIRef('http://o.org/same_continent'),) london

paris (rdflib.term.URIRef('http://o.org/same_country'),) london
0


In [11]:
import json

def load_json(frame_path):
    chat_file = open(frame_path, encoding='utf-8')
    chat = json.load(chat_file)
    return chat

final_vector = []
def prepare_frames_vector(chat):
    dst_city = or_city = str_date = end_date = ""
    or_city_diff = dst_city_diff = str_date_diff = end_date_diff = 0
    for t in chat["turns"]:
        if "text" in t:
            prev_or_city = or_city
            prev_dst_city = dst_city
            prev_str_date = str_date
            prev_end_date = end_date
            
            for f in t["labels"]["frames"]:
                #print(f["info"])
                info = f["info"]
                if "or_city" in info:
                    or_city = info["or_city"][0]["val"]
                if "dst_city" in info:
                    dst_city = info["dst_city"][0]["val"]
                if "str_date" in info:
                    str_date = info["str_date"][0]["val"]
                if "end_date" in info:
                    end_date = info["end_date"][0]["val"]

            if len(prev_or_city) > 1:
                or_city_diff = geographic_difference(g, prev_or_city, or_city)
            if len(prev_dst_city) > 1:
                dst_city_diff = geographic_difference(g, prev_dst_city, dst_city)
                
            if prev_str_date is not None and len(prev_str_date) > 1:
                str_date_diff = date_difference(str_date, prev_str_date)
            if prev_end_date is not None and len(prev_end_date) > 1:
                end_date_diff = date_difference(end_date, prev_end_date)

            print()
            print(t["author"])
            print(t["text"])
            print(or_city, dst_city, str_date, end_date)
            print(or_city_diff, dst_city_diff, str_date_diff, end_date_diff)
            
            final_vector.append([t["author"], t["text"], or_city_diff+dst_city_diff, str_date_diff+end_date_diff])
    
    return final_vector
                    
#chat = load_json("../Frames-dataset/chats/00d28996-ed48-4f09-800e-677bae20547f.json")
#chat = load_json("../Frames-dataset/chats/0ac64405-3e57-4f5a-96d2-9c564e78d6e5.json")
chat = load_json("../Frames-dataset/chats/be1e346a-6892-4d34-acc8-dfd0a3a935c4.json")
print()
print(prepare_frames_vector(chat))
    



user
hi i want to go on vacation i leave from cancun
cancun   
0 0 0 0

wizard
Ok! Where would you like to go?
cancun   
0 0 0 0

user
im thinking of Santos
cancun Santos  
0 0 0 0

wizard
Sure thing! Any dates in mind?
cancun Santos  
0 0 0 0

user
i want to leave on the 15th
cancun Santos 15 
0 0 0 0
15
15

wizard
The 15th of August? I can do that for you.
cancun Santos 15 
0 0 0 0
august 15
15

user
perfect!
cancun Santos august 15 
0 0 255 0
august 15
august 15

wizard
I have a trip available from August 15-August 26. Does this work for you?
cancun Santos august 15 august 26
0 0 0 0
august 15
august 15
None
august 26

user
yes it does! what type of hotel?
cancun Santos august 15 None
0 0 0 0
august 15
august 15

wizard
Great! The hotel I've found for you has a 4 star rating.
cancun Santos august 15 None
0 0 0 0
august 15
august 15

user
very nice! whats the pricing like?
cancun Santos august 15 None
0 0 0 0
august 15
august 15

wizard
The price for that package would be 2877.68US